In [ ]:
import pandas as pd

symbol = 'iTSLAUSDT'

csv_path = 'gap_study/MCD_1d_quotes.csv'
df_price = pd.read_csv(csv_path)

df_price = df_price[df_price['symbol'] == symbol]
df_price['price'] = df_price['indexMedianPrice']
df_price['time'] = (df_price['time'] * 1000).astype(int)

columns_to_drop = ['symbol', 'fastVolatility', 'type', 'slowVolatility', 'indexMedianPrice']
df_price.drop(columns=columns_to_drop, inplace=True)

# df_price['time'] = pd.to_datetime(df_price['time'], unit='s', utc=True)

df_price

In [ ]:
csv_path_trades = '/Users/theoverdelhan/Documents/TRADING/MYR/Injective/FETCHING_DATA/trades_tesla.csv'
df_trades = pd.read_csv(csv_path_trades)

trades_df = pd.DataFrame(df_trades)

trades_df.sort_values(by='executedAt', ascending=True, inplace=True)

trades_df['time'] = trades_df['executedAt']
trades_df['entry_price'] = trades_df['executionPrice'] / 10**6
trades_df['quantity'] = trades_df['executionQuantity']
trades_df['direction'] = trades_df['tradeDirection']
trades_df['fee'] = trades_df['fee'] / 10**6

# trades_df['time'] = pd.to_datetime(trades_df['time'], unit='ms', utc=True)

columns_to_drop = ['executedAt', 'executionPrice', 'executionQuantity', 'executionMargin' ,'tradeDirection', 'fee']
trades_df.drop(columns=columns_to_drop, inplace=True)
trades_df

In [ ]:
# df_price = df_price.sort_values('time')
# trades_df = trades_df.sort_values('time')

merged = pd.merge_asof(trades_df, df_price, on='time', direction='nearest')
merged = merged.dropna()
merged.head()

,symbol,indexPrice,spread,time,entry_price,quantity,direction,ask_price,bid_price,mid_price,price
0,iTSLAUSDT,264.700,-0.005667,1743640126993,263.20,4.409,buy,260.13,260.06,260.095,260.095
1,iTSLAUSDT,270.925,0.003968,1743673517951,272.00,0.200,sell,271.46,270.88,271.170,271.170
2,iTSLAUSDT,268.910,-0.003049,1743675482551,268.09,0.200,buy,269.06,268.76,268.910,268.910
3,iTSLAUSDT,267.375,0.003048,1743675754682,268.19,0.400,sell,267.46,267.15,267.305,267.305
4,iTSLAUSDT,270.615,0.000573,1743687243772,270.77,40.000,sell,270.71,270.53,270.620,270.620


In [ ]:
min_time_price = df_price['time'].min()
max_time_price = df_price['time'].max()

count_in_interval = df_trades[(df_trades['executedAt'] >= min_time_price) & (df_trades['executedAt'] <= max_time_price)]
print(f"Nombre de timestamps dans l'intervalle de df_price : {len(count_in_interval)}")

Nombre de timestamps dans l'intervalle de df_price : 181


In [ ]:
merged['spread_exec'] = ((merged['entry_price'] - merged['price'])) / ((merged['entry_price'] + merged['price']) / 2) * 100
merged.head()

,symbol,indexPrice,spread,time,entry_price,quantity,direction,ask_price,bid_price,mid_price,price,spread_exec
0,iTSLAUSDT,264.700,-0.005667,1743640126993,263.20,4.409,buy,260.13,260.06,260.095,260.095,1.186711
1,iTSLAUSDT,270.925,0.003968,1743673517951,272.00,0.200,sell,271.46,270.88,271.170,271.170,0.305613
2,iTSLAUSDT,268.910,-0.003049,1743675482551,268.09,0.200,buy,269.06,268.76,268.910,268.910,-0.305400
3,iTSLAUSDT,267.375,0.003048,1743675754682,268.19,0.400,sell,267.46,267.15,267.305,267.305,0.330535
4,iTSLAUSDT,270.615,0.000573,1743687243772,270.77,40.000,sell,270.71,270.53,270.620,270.620,0.055413


In [ ]:
import plotly.graph_objects as go

merged['datetime'] = pd.to_datetime(merged['time'], unit='ms')

df_buy = merged[merged['direction'] == 'buy']
df_sell = merged[merged['direction'] == 'sell']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_buy['datetime'],
    y=df_buy['spread_exec'],
    mode='markers',
    name='Buy',
    marker=dict(color='green', size=4),
    hovertemplate='Time: %{x}<br>Spread Exec: %{y:.4f}%<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=df_sell['datetime'],
    y=df_sell['spread_exec'],
    mode='markers',
    name='Sell',
    marker=dict(color='red', size=4),
    hovertemplate='Time: %{x}<br>Spread Exec: %{y:.4f}%<extra></extra>'
))

fig.update_layout(
    title='Spread à l’exécution dans le temps',
    xaxis_title='Heure (UTC)',
    yaxis_title='Spread Exec (%)',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go


timestamp_to_plot = 1743689007065

window = 2 * 1000


# Filtrage autour de la timestamp
df_window = merged[
    (merged['time'] >= timestamp_to_plot - window) &
    (merged['time'] <= timestamp_to_plot + window)
].copy()  # copy pour éviter les SettingWithCopyWarning

# Calcul du prix de référence (mid_price au début de la fenêtre)
price_ref = df_window.iloc[0]['price']
df_window['spread'] = df_window['price'] - price_ref

# Création du graphique
fig = go.Figure()

# Trace mid_price
fig.add_trace(go.Scatter(
    x=df_window['time'],
    y=df_window['price'],
    mode='lines',
    name='Mid Price'
))

# Point rouge à la timestamp cible
closest_idx = df_window['time'].sub(timestamp_to_plot).abs().idxmin()
closest_time = df_window.loc[closest_idx, 'time']
closest_price = df_window.loc[closest_idx, 'price']

fig.add_trace(go.Scatter(
    x=[closest_time],
    y=[closest_price],
    mode='markers',
    marker=dict(color='red', size=10),
    name='Target Timestamp'
))

# Ajout de la courbe de spread
fig.add_trace(go.Scatter(
    x=df_window['time'],
    y=df_window['spread'],
    mode='lines',
    name='Spread (mid_price - price_ref)',
    yaxis='y2',
    line=dict(dash='dot', color='orange')
))

# Layout avec un deuxième axe Y pour le spread
fig.update_layout(
    title=f"Mid Price et Spread autour de la timestamp {timestamp_to_plot}",
    xaxis_title="Time (ms)",
    yaxis=dict(title="Mid Price"),
    yaxis2=dict(title="Spread", overlaying='y', side='right', showgrid=False),
    legend=dict(x=0, y=1),
    height=600
)

fig.show()